In [1]:
import json
from glob import glob
from multiprocessing import Pool
from pathlib import Path
import os
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from stexls.util import file_watcher
from pickle import dumps, loads, dump, load

from stexls.util.latex import parser

from stexls import *

In [2]:
%cd ~/Downloads

/home/marian/Downloads


In [3]:
def parse_files(root=Path.home()/'Downloads/smglom', limit: int = -1):
    ww = file_watcher.WorkspaceWatcher(root, filter='*.tex')
    changes = ww.update()
    with Pool() as pool:
        parsed_files = list(tqdm(pool.imap_unordered(parse, list(changes.created | changes.modified)[:limit])))
    return parsed_files

In [ ]:
use_cache = False
cachefile = Path('/tmp/cache.bin')
if cachefile.is_file() and use_cache:
    with open(cachefile, 'rb') as fd:
        parsed_files = load(fd)
else:
    parsed_files = parse_files()
    with open(cachefile, 'w+b') as fd:
        dump(parsed_files, fd)

line 5:0 mismatched input '<EOF>' expecting {'{', '[', ']', '=', ',', MATH_ENV, TEXT, 'begin', 'end', INLINE_ENV_NAME}
line 7:0 mismatched input '<EOF>' expecting {'{', '[', ']', '=', ',', MATH_ENV, TEXT, 'begin', 'end', INLINE_ENV_NAME}


In [ ]:
for pf in parsed_files:
    if pf.errors:
        print(pf.errors)
        print()
        print('-'*80)
        print()

In [ ]:
with Pool() as pool:
    compiled_files = pool.map(StexObject.compile, tqdm(parsed_files))

In [ ]:
ok = list(filter(None, compiled_files))

In [ ]:
links, resolved_references = index_and_link(ok)

In [ ]:
for id, refs in resolved_references.items():
    for ref in refs:
        if isinstance(ref, DefSymbol):
            if ref.noverb or ref.noverbs:
                print(ref, ref.noverbs)

In [ ]:
resolved_references